# Lowpass filter instead of Kalman filter for the PIT

In [ ]:
# %load imports.py
## Local packages:

%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

## External packages:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import numpy as np
import os
import matplotlib.pyplot as plt
#if os.name == 'nt':
#    plt.style.use('presentation.mplstyle')  # Windows

import plotly.express as px 
import plotly.graph_objects as go

import seaborn as sns
import sympy as sp
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex
from src.substitute_dynamic_symbols import run, lambdify

import pyro

import sklearn
import pykalman
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

from scipy.integrate import solve_ivp

## Local packages:
from src.data import mdl

from src.symbols import *
from src.parameters import *
import src.symbols as symbols
from src import prime_system
from src.models import regression
from src.visualization.plot import track_plot
from src.equation import Equation

## Load models:
# (Uncomment these for faster loading):
import src.models.vmm_abkowitz  as vmm_abkowitz 

In [ ]:
from src.data import kalman_filter
from src.data.lowpass_filter import lowpass_filter
import scipy.stats as stats

In [ ]:
id=22774
df, units, meta_data = mdl.load(id=id)
df_raw = df.copy()
df.index = df.index.total_seconds()
df.index-=df.index[0]

In [ ]:
def df_filter(df_gradient, cutoff=1, order=5):
    
    ts = np.mean(np.diff(df_gradient.index))
    fs = 1/ts
    
    df_lowpass = pd.DataFrame(columns=df_gradient.columns, index=df_gradient.index)
    for key,value in df_gradient.items():
        df_lowpass[key] = lowpass_filter(data=value, cutoff=cutoff, fs=fs, order=order)
        
    return df_lowpass

def lowpass(df_,cutoff=1, order=5):
    
    df_gradient = df_.copy()
    df_gradient.index=df.index
    
    df_gradient[['x0_filtered','y0_filtered','psi_filtered']] = df_filter(df_gradient[['x0','y0','psi']], cutoff=cutoff, order=order)
    df_gradient['x01d'] = np.gradient(df_gradient['x0_filtered'], df_gradient.index)
    df_gradient['y01d'] = np.gradient(df_gradient['y0_filtered'], df_gradient.index)
    df_gradient['z01d_gradient'] = np.gradient(df_gradient['z0'], df_gradient.index)
    
    df_gradient['r'] = df_gradient['psi1d'] = np.gradient(df_gradient['psi_filtered'], df_gradient.index)
    
    df_gradient[['x01d','y01d','r']] = df_filter(df_gradient[['x01d','y01d','r']], cutoff=cutoff, order=order)
    df_gradient['x02d'] = np.gradient(df_gradient['x01d'], df_gradient.index)
    df_gradient['y02d'] = np.gradient(df_gradient['y01d'], df_gradient.index)
    df_gradient['z02d_gradient'] = np.gradient(df_gradient['z01d_gradient'], df_gradient.index)
    df_gradient['r1d'] = df_gradient['psi2d'] = np.gradient(df_gradient['r'], df_gradient.index)
    
    df_gradient[['x02d','y02d','r1d']] = df_filter(df_gradient[['x02d','y02d','r1d']], cutoff=cutoff, order=order)
    
    
    return df_gradient

In [ ]:
df_lowpass = lowpass(df_=df, cutoff=1, order=1)

df_resample = df_raw.resample('0.5S').mean()
t_ = df_resample.index.total_seconds()
df_resample['x01d'] = np.gradient(df_resample['x0'], t_)
df_resample['y01d'] = np.gradient(df_resample['y0'], t_)
df_resample['r'] = np.gradient(df_resample['psi'], t_)
df_resample['x02d'] = np.gradient(df_resample['x01d'], t_)
df_resample['y02d'] = np.gradient(df_resample['y01d'], t_)
df_resample['r1d'] = np.gradient(df_resample['r'], t_)
df_resample.index=df_resample.index.total_seconds()
df_resample.index-=df_resample.index[0]


df_rolling = df.sort_index(ascending=False).rolling(100).mean().sort_index(ascending=True)
t_=df_rolling.index
df_rolling['x01d'] = np.gradient(df_rolling['x0'], t_)
df_rolling['y01d'] = np.gradient(df_rolling['y0'], t_)
df_rolling['r'] = np.gradient(df_rolling['psi'], t_)
df_rolling['x02d'] = np.gradient(df_rolling['x01d'], t_)
df_rolling['y02d'] = np.gradient(df_rolling['y01d'], t_)
df_rolling['r1d'] = np.gradient(df_rolling['r'], t_)

states = [
     ['x0','x01d','x02d'],
     ['y0','y01d','y02d'],
     ['psi','r','r1d']
]

for dof in states:
    fig,axes=plt.subplots(ncols=3)
    fig.set_size_inches(15,4)
    for state,ax in zip(dof,axes):
        ax.set_title(state)
        #df_lowpass_gradient.plot(y=state, ax=ax, label='lowpass gradient')
        df_lowpass.plot(y=state, ax=ax, label='lowpass')
        df_resample.plot(y=state, ax=ax, label='resample', alpha=0.5)
        df_rolling.plot(y=state, ax=ax, label='rolling', alpha=0.5)

In [ ]:
kalman_filter.transform_to_ship(df_lowpass, include_unfiltered=False)